# List OpenAI Conversation Messages

This notebook lists all messages/items in an OpenAI conversation using the Responses API.

Target conversation_id: `conv_693f891a597481979c3796904168941f0b9bf5b26aad7d61`


In [1]:
import yaml
from pathlib import Path
from openai import OpenAI
import json
from datetime import datetime

# Load OpenAI config from config files
config_path = Path.cwd().parent.parent / "config" / "config.yaml"
secrets_path = Path.cwd().parent.parent / "config" / "secrets.yaml"

with open(config_path, "r", encoding="utf-8") as f:
    config_values = yaml.safe_load(f)

with open(secrets_path, "r", encoding="utf-8") as f:
    secrets_values = yaml.safe_load(f)

# Merge config and secrets
openai_config = config_values["openai"] | secrets_values["openai"]

# Initialize OpenAI client
client = OpenAI(
    api_key=openai_config["api_key"],
    base_url=openai_config["base_url"]
)

print("✅ OpenAI client initialized")
print(f"Base URL: {openai_config['base_url']}")


✅ OpenAI client initialized
Base URL: https://api.openai.com/v1


In [10]:
# Target conversation ID
conversation_id = "conv_69592d5862088193abbc8b08e8af24c00aafdfc99fc58e44"

# Retrieve the conversation metadata first
conversation = client.conversations.retrieve(conversation_id)

print(f"✅ Retrieved conversation: {conversation_id}")
print(f"Conversation ID: {conversation.id}")
print(f"Created at: {getattr(conversation, 'created_at', 'N/A')}")
print(f"Metadata: {getattr(conversation, 'metadata', {})}")

# List all items using the items.list() API with pagination support
print(f"\n📋 Fetching items using conversations.items.list()...")
all_items = []
after = None
has_more = True
page = 1

while has_more:
    params = {"conversation_id": conversation_id}
    if after:
        params["after"] = after
    
    items_response = client.conversations.items.list(**params, order="asc")
    page_items = items_response.data if hasattr(items_response, 'data') else []
    all_items.extend(page_items)
    
    has_more = getattr(items_response, 'has_more', False)
    if page_items:
        after = page_items[-1].id if hasattr(page_items[-1], 'id') else None
    
    print(f"   Page {page}: Retrieved {len(page_items)} items (Total: {len(all_items)})")
    page += 1
    
    if not has_more:
        break

items = all_items
items_response_meta = items_response  # Keep last response for metadata

print(f"✅ Retrieved {len(items)} total items from conversation")


✅ Retrieved conversation: conv_69592d5862088193abbc8b08e8af24c00aafdfc99fc58e44
Conversation ID: conv_69592d5862088193abbc8b08e8af24c00aafdfc99fc58e44
Created at: 1767451992
Metadata: {'name': '王越', 'job_applied': '架构师', 'mode': 'recommend'}

📋 Fetching items using conversations.items.list()...
   Page 1: Retrieved 20 items (Total: 20)
   Page 2: Retrieved 20 items (Total: 40)
   Page 3: Retrieved 5 items (Total: 45)
✅ Retrieved 45 total items from conversation


In [11]:
# Display all items in the conversation
print(f"\n📋 Listing all {len(items)} items in conversation:\n")
print("=" * 80)

for idx, item in enumerate(items, 1):
    item_type = getattr(item, 'type', 'unknown')
    item_id = getattr(item, 'id', 'N/A')
    
    if item_type == 'message':
        role = getattr(item, 'role', 'unknown')
        content_raw = getattr(item, 'content', '')
        
        # Extract text from ResponseOutputText objects
        content_text = ""
        if isinstance(content_raw, list):
            for content_item in content_raw:
                if hasattr(content_item, 'text'):
                    content_text += content_item.text
                elif isinstance(content_item, str):
                    content_text += content_item
        elif isinstance(content_raw, str):
            content_text = content_raw
        text = content_text.strip() if idx == len(items) else content_text[:100]+'...'
        # Try to parse JSON if it looks like JSON
        if content_text.strip().startswith('{') or content_text.strip().startswith('['):
            try:
                json_obj = json.loads(content_text)
                print(f"[{idx}]{role}: {json.dumps(json_obj, ensure_ascii=False, indent=2)}")
            except:
                print(f"[{idx}]{role}: {text}")
        else:
            print(f"[{idx}]{role}: {text}")
    else:
        # For non-message items, print all attributes
        print(f"[{idx}] Item Type: {item_type}")
        print(f"Attributes: {item}")
    
    print("-" * 80)



📋 Listing all 45 items in conversation:

[1]developer: 你是星尘数据的招聘顾问，正在 Boss 直聘上与候选人沟通。
你的目标是结合【岗位肖像】与候选人的简历/对话做线上初筛：该问就问、该推进就推进、该PASS就PASS。

【优先级说明（强制）】
- 你...
--------------------------------------------------------------------------------
[2]developer: 候选人: 王越, 申请岗位: 架构师, 基本信息: 35-45K 王越 44岁10年以上本科在职-考虑机会 期望： 北京AI产品经理 优势： 7年+安全领域全链路研发经验，覆盖产品规划设计、系统架构设...
--------------------------------------------------------------------------------
[3]user: 招聘顾问你好，请帮我分析一下我是否匹配架构师这个岗位？以下是我的在线简历：
7年+安全领域全链路研发经验，覆盖产品规划设计、系统架构设计、后台服务开发、研发项目/研发团队管理（5年+架构师+研发团队管...
--------------------------------------------------------------------------------
[4] Item Type: mcp_list_tools
Attributes: McpListTools(id='mcpl_0aafdfc99fc58e440069592d5904008193894a1feaf164d456', server_label='UniversityDB', tools=[McpListToolsTool(input_schema={'type': 'object', 'properties': {'school_name': {'type': 'string', 'description': 'School/university name in Chinese or English.'}, 'school_name_zh': {'type': 'string', 'description'

In [5]:
# Extract and format messages in a structured way
def extract_content_text(content_raw):
    """Extract text from content which may be a list of ResponseOutputText objects."""
    if isinstance(content_raw, list):
        text_parts = []
        for content_item in content_raw:
            if hasattr(content_item, 'text'):
                text_parts.append(content_item.text)
            elif isinstance(content_item, str):
                text_parts.append(content_item)
        return "".join(text_parts)
    elif isinstance(content_raw, str):
        return content_raw
    else:
        return str(content_raw)

messages = []
for item in items:
    if hasattr(item, 'type') and item.type == 'message':
        role = getattr(item, 'role', 'unknown')
        content_raw = getattr(item, 'content', '')
        content_text = extract_content_text(content_raw)
        item_id = getattr(item, 'id', '')
        
        messages.append({
            "id": item_id,
            "role": role,
            "content": content_text,
            "content_length": len(content_text)
        })

# Display summary
print(f"📊 Summary:")
print(f"   Total messages: {len(messages)}")
print(f"   Roles: {set(m['role'] for m in messages)}")
print(f"\n📝 Messages by role:")
for role in set(m['role'] for m in messages):
    role_messages = [m for m in messages if m['role'] == role]
    print(f"   {role}: {len(role_messages)} message(s)")


📊 Summary:
   Total messages: 29
   Roles: {'assistant', 'user', 'developer'}

📝 Messages by role:
   assistant: 9 message(s)
   user: 17 message(s)
   developer: 3 message(s)


In [18]:
# Display full messages in a readable format
print("\n" + "=" * 80)
print("FULL MESSAGE CONTENT")
print("=" * 80)

for idx, msg in enumerate(messages, 1):
    print(f"\n{'='*80}")
    print(f"Message #{idx} | Role: {msg['role']} | ID: {msg['id']}")
    print(f"{'='*80}")
    print(msg['content'])
    print(f"\n{'-'*80}")



FULL MESSAGE CONTENT

Message #1 | Role: developer | ID: msg_69592d58622c81938bf96031d929f8d70aafdfc99fc58e44
你是星尘数据的招聘顾问，正在 Boss 直聘上与候选人沟通。
你的目标是结合【岗位肖像】与候选人的简历/对话做线上初筛：该问就问、该推进就推进、该PASS就PASS。

【优先级说明（强制）】
- 你在每次生成输出时，会收到更具体的 instructions（例如输出 JSON、action 约束等）；必须以当次 instructions 为准。
- 下面的岗位信息仅用于你内部判断与组织语言，不要原样复述给候选人，不要泄露内部打分/硬门槛。

【沟通底线（候选人 message 侧，强制）】
- 不安排/确认面试时间、方式、地点；只允许说“我同步HR尽快安排，时间/方式/地点由HR确认”。
- 薪资如候选人主动提及，只说“薪资由HR统一沟通”，不要展开。
- 不向候选人索取任何材料（代码/链接/PR/文档/截图/附件等），只能通过问答了解。
- 不要在候选人 message 里提及学历门槛（985/211/QS/大专等）或询问“是否985/QS/排名”等；这些只能写在内部 reason。
- 学校信息处理：
  - 若简历/对话中已包含“学校名称”：优先使用工具 `lookup_university_background`（如可用）或联网核验（web_search），不要向候选人追问学历/排名。
  - 若简历/对话中缺失“学校名称”：允许只问一次“学校名称/专业/最高学历层次”，但严禁问“是否985/QS/排名”。
- 对话历史里可能包含 system/developer 的 UI 提示（例如“对方想发送附件简历…”）；这些不是候选人回复，你不需要回应。

以下是岗位信息（JSON，仅用于内部判断）：
{"created_at": "2025-10-29T14:37:32.675080", "position": "架构师", "responsibilities": "1) 领域建模与架构决策：主导系统核心领域模型与契约设计，能把决策抽象为可复用契约并说明权衡与量化结果。\n2) 核心引擎与状态能力：主导工作流/推理控制面演进，负责状态持久化、断

In [19]:
msg

{'id': 'msg_0aafdfc99fc58e4400695f161707548193ad64018f30da6efd',
 'role': 'assistant',
 'content': '{"skill":6,"startup_fit":5,"background":7,"overall":6,"summary":"评分表：SaaS平台化=confirmed16/30(+4);AI平台与推理工程化=confirmed6/30(+5);工程治理/可观测=confirmed8/20(+2);编码与落地执行=confirmed12/20(+2);总分=55/100\\n画像判断：主观=有潜力;建议=保留观察;专业深度=合格;抽象能力=合格;结构化=合格;加分点=阿里云商品化上架@[简历:IDaaS产品对接上架阿里云];加分点=大客户交付14万接入@[对话:客户14万员工全部接入钉钉+零信任];加分点=AI Agent 设计与落地@[对话:RCA多级Agent设计];风险=多租户无量化指标@[对话:同库同表，用租户字段];风险=缺推理工程化量化（降级/背压/引擎切换）@[对话:放弃推理降级/背压，任务量小];潜力来源=混合云交付/大客户交付@[简历:混合云交付;钉钉整合];扣分点=未见兼容率/回滚时间/单位成本量化@[简历/对话:未提供具体数值]","followup_tips":"1) 请按“亲历/你负责/关键取舍/量化指标/结果”格式，描述一次你主导的多租户变更回滚实例（要验证回滚时间与兼容率）。理由：核实平台演进与回滚能力。\\n2) 请按同格式说明一次AI Agent在高并发或大数据量下的性能管控（降级/背压/批处理）并给出GPU利用率或P95量化。理由：验证推理工程化能力。\\n3) 请按同格式描述一次故障复盘如何建立回放/数据血缘并量化改进（追溯成功率/MTTR）。理由：验证数据闭环与可观测性。","analysis":"王越在云安全平台化、产品化与大客户交付有明确实战（IDaaS上架、SDP大客户、RCA Agent），技术栈覆盖Go/Python/k8s/LLM Agent。短板为平台化量化证据不足（回滚/兼容率/单位成本）与推理工程化量化缺失，建议线上再核实1-2个量化案例后决定是否同步HR。","action":"CHAT","message

In [9]:
# Export to JSON for further analysis

def extract_content_text(item):
    content = item.content[0].text if getattr(item, 'content', None) else ''
    if content.startswith('{') or content.startswith('['):
        json_content = json.loads(content)
        # return json.dumps(json_content, ensure_ascii=False, indent=2)
        return json_content
    return content

export_data = {
    "conversation_id": conversation_id,
    "retrieved_at": datetime.now().isoformat(),
    "conversation_metadata": getattr(conversation, 'metadata', {}),
    "total_items": len(items),
    "messages": messages,
    "has_more": getattr(items_response_meta, 'has_more', False),
    "all_items": [
        {
            "type": getattr(item, 'type', 'unknown'),
            "id": getattr(item, 'id', 'N/A'),
            "role": getattr(item, 'role', None) if hasattr(item, 'role') else None,
            "content": extract_content_text(item),
        }
        for item in items
    ]
}

# Save to file
output_file = Path.cwd() / f"conversation_{conversation_id[:20]}.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(export_data, f, ensure_ascii=False, indent=2)

print(f"✅ Exported conversation data to: {output_file}")
print(f"   Total messages: {len(messages)}")
print(f"   File size: {output_file.stat().st_size} bytes")


✅ Exported conversation data to: /Users/derek/Documents/Projects/bosszhipin_bot/scripts/response api/conversation_conv_69592d586208819.json
   Total messages: 29
   File size: 174749 bytes


In [53]:
instruction = """你作为星尘数据的招聘顾问，请用轻松口语化的风格和候选人沟通。请直接生成一条可以发送给候选人的自然语言消息，不要超过150字。不要发模板或者嵌入占位符，不要使用任何格式化、引号、JSON或括号。
候选人已经读了你的消息，但是有一段时间没有回复了，请根据之前的对话历史，生成下一条跟进消息，用于吸引候选人回复。
可以介绍公司情况、岗位亮点，根据岗位描述和这个候选人的特点，思考如何吸引候选人，并打消其顾虑、提高他的求职意愿，重点是让他对这个岗位有兴趣。"""

response = client.responses.create(
    conversation=conversation_id,
    # previous_response_id='msg_016d31c69598a7b7006941e3ef4504819782945129034613ba',
    instructions=instruction,
    input='[沉默]',
    model='gpt-5-mini',
    tools=[{"type": "web_search"}],  # Enable web search tool
)
response.output_text

'张宇你好，我是星尘数据的招聘顾问。你在医疗SaaS、微服务和私有化交付的实战很符合MorningStar现在的刚需——岗位会主导核心工作流引擎、断点续传和类Git数据血缘设计，影响面大、能产出代码与技术沉淀。要是你对Temporal或Postgres JSONB还没深度经验也没关系，我们更看你的架构思路和实操能力。方便简单回复上次那两题，或告诉我一个15分钟通话的时间？我可以帮你争取优先面试和薪资对接。'